در پیشبینی بر اساس همکاری در واقع ما کاربرهای شبیه به یکدیگر را پیدا میکنیم

 و فیلم هایی که نسبت به آنها بازخورد خوبی داشتند را به دیگری پیشنهاد میدهیم،
 
  احتمالا نفر دوم هم خوشش می آید! با کتابخانه ها شروع میکنیم

In [1]:
import pandas as pd
import numpy as np
from math import sqrt

دیتافریم ها را ایجاد میکنیم و ستون های به درد نخور را دور میریزیم

In [2]:
movie_df = pd.read_csv('movies.csv')
rating_df = pd.read_csv('ratings.csv')

movie_df = movie_df.drop("genres", axis=1)
rating_df = rating_df.drop("timestamp", axis=1)

movie_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


با استفاده از ریجکس سال تولید را از عنوان فیلم استخراج میکنیم و در ستونی جداگانه ذخیره اش میکنیم

In [3]:
titles = movie_df["title"]
movie_df["year"] = movie_df["title"].str.extract('(\(\d\d\d\d\))',expand=False)
movie_df["year"] = movie_df["year"].str.extract('(\d\d\d\d)',expand=False)
movie_df["title"] = movie_df["title"].str.replace('(\(\d\d\d\d\))', '')
movie_df["title"] = movie_df["title"].apply(lambda x : x.strip())
movie_df.head()

,movieId,title,year
0,1,Toy Story (1995),1995
1,2,Jumanji (1995),1995
2,3,Grumpier Old Men (1995),1995
3,4,Waiting to Exhale (1995),1995
4,5,Father of the Bride Part II (1995),1995


In [4]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


یوزر فرضی را میسازیم

In [5]:
user = [
            {'title':'Babe (1995)', 'rating':5},
            {'title':'Pocahontas (1995)', 'rating':4},
            {'title':'Indian in the Cupboard, The (1995)', 'rating':3},
            {'title':"Mr. Holland's Opus (1995)", 'rating':5},
            {'title':'NeverEnding Story III, The (1994)', 'rating':2}
        ]
inputMovies = pd.DataFrame(user)
inputMovies

,title,rating
0,Babe (1995),5
1,Pocahontas (1995),4
2,"Indian in the Cupboard, The (1995)",3
3,Mr. Holland's Opus (1995),5
4,"NeverEnding Story III, The (1994)",2


یک دیتافریم جدید از فیلم هایی که یوزر ساختگی داریم میسازیم

In [6]:
inputId = movie_df[movie_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('year', axis=1)
inputMovies

,movieId,title,rating
0,34,Babe (1995),5
1,48,Pocahontas (1995),4
2,60,"Indian in the Cupboard, The (1995)",3
3,62,Mr. Holland's Opus (1995),5
4,126,"NeverEnding Story III, The (1994)",2


یک دیتافریم از یوزرها و فیلم ها به همراه امتیازی که هر یوزر به آن فیلم داده است میسازیم،

در این دیتافریم ما صرفا فیلم هایی را داریم که از کاربر فرضی دریافت کردیم

In [7]:
userSubset = rating_df[rating_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
308,4,126,1.0
518,5,34,4.0
582,6,34,4.0
591,6,60,4.0
593,6,62,4.0


In [8]:
userSubsetGroup = userSubset.groupby(['userId'])

بر اساس آیدی هر یوزر اطلاعات فیلم ها را دسته بندی میکنیم و بر اساس آیدی هر فیلم مرتب سازی میکنیم

In [9]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[0:3]

[((27,),
        userId  movieId  rating
  4061      27       34     5.0
  4062      27       48     4.0
  4063      27       60     3.0
  4064      27       62     5.0
  4065      27      126     2.0),
 ((40,),
        userId  movieId  rating
  5561      40       34     5.0
  5564      40       48     2.0
  5565      40       60     4.0
  5566      40       62     5.0
  5573      40      126     3.0),
 ((6,),
       userId  movieId  rating
  582       6       34     4.0
  591       6       60     4.0
  593       6       62     4.0
  612       6      126     3.0)]

In [10]:
userSubsetGroup = userSubsetGroup[0:100]

یک دیکشنری میسازیم که قرار است در آن آیدی هر فرد را کلید دیکشنری و شباهت آن فرد با یوزر فرضی ما را ولیو آن قرار میدهیم.

صورت و مخرج فرمولی که برای شباهت استفاده میکنیم را ایجاد میکنیم و با یک حلقه برای هر فرد میزان شباهت را نسبت میدهیم

In [11]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group = group.sort_values(by='movieId')
    
    inputMovies = inputMovies.sort_values(by='movieId')
    
    nRatings = len(group)
    
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()

    # x1, x2, x3 are the parameters in the formula
    x1 = sum([i ** 2 for i in tempRatingList]) - pow(sum(tempRatingList), 2) / float(nRatings)
    x2 = sum([i ** 2 for i in tempGroupList]) - pow(sum(tempGroupList), 2) / float(nRatings)
    x3 = sum(i * j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList) * sum(tempGroupList) / float(nRatings)
    
    if x1 != 0 and x2 != 0:
        pearsonCorrelationDict[name] = x3 / sqrt(x1 * x2)
    else:
        pearsonCorrelationDict[name] = 0

pearsonCorrelationDict

{(27,): 1.0,
 (40,): 0.5588235294117645,
 (6,): 0.7777777777777778,
 (117,): 0.5222329678670935,
 (169,): 0.48420012470625223,
 (234,): 0.4,
 (274,): 0.15289415743128767,
 (323,): -0.6363636363636364,
 (395,): 0.4472135954999579,
 (43,): 0,
 (68,): -0.6286185570937086,
 (103,): 0.0,
 (160,): 0.9958705948858224,
 (235,): 0.8660254037844355,
 (240,): 0.5,
 (372,): 0.8660254037844355,
 (380,): 0.0,
 (381,): 0.7559289460184498,
 (414,): 0.8660254037844355,
 (436,): 1.0,
 (480,): 0.9707253433941521,
 (525,): 0.49999999999998934,
 (584,): -0.866025403784439,
 (604,): 0,
 (19,): 1.0,
 (20,): -1.0,
 (35,): 0,
 (38,): 0,
 (58,): 0,
 (60,): 0,
 (64,): 1.0,
 (84,): 0,
 (93,): 1.0,
 (109,): 0,
 (111,): -1.0,
 (132,): -1.0,
 (136,): 0,
 (140,): 0,
 (144,): -1.0,
 (156,): 0,
 (170,): 1.0,
 (174,): 0,
 (177,): 0,
 (181,): 0,
 (200,): 0,
 (216,): 1.0,
 (217,): 1.0,
 (226,): 1.0,
 (243,): 1.0,
 (287,): 1.0,
 (288,): 1.0,
 (292,): 0,
 (304,): 1.0,
 (314,): 1.0,
 (330,): 0,
 (357,): 0,
 (373,): 1.0,
 (40

از دیکشنری که ساختیم یک دیتافریم ایجاد میکنیم که شامل آیدی هر یوزر و میزان شباهت آن با یوزر فرضی ما است

In [12]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,1.000000,"(27,)"
1,0.558824,"(40,)"
2,0.777778,"(6,)"
3,0.522233,"(117,)"
4,0.484200,"(169,)"


بر اساس میزان شباهت یوزرها را از شباهت زیاد به کم مرتب سازی میکنیم

In [13]:
topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
0,1.0,"(27,)"
19,1.0,"(436,)"
30,1.0,"(64,)"
32,1.0,"(93,)"
40,1.0,"(170,)"


آیدی هر یوزر را از حالت تاپل به یک عدد تبدیل میکنیم

In [14]:
userId_list = list(topUsers["userId"])
new_userId_list = []
for id in userId_list:
    new_userId_list.append(id[0])

topUsers = topUsers.drop("userId", axis=1)

topUsers["userId"] = new_userId_list

topUsers.head()

,similarityIndex,userId
0,1.0,27
19,1.0,436
30,1.0,64
32,1.0,93
40,1.0,170


دیتافریمی که از امتیازات فیلم ها داشتیم را به کار میبریم و با دیتافریمی که از یوزرهای مشابه داشتیم یک ترکیب میکنیم و یک دیتافریم جدید میسازیم

In [15]:
topUsersRating = pd.merge(topUsers, rating_df, on='userId', how='inner')
topUsersRating.head()


,similarityIndex,userId,movieId,rating
0,1.0,27,1,3.0
1,1.0,27,2,4.0
2,1.0,27,34,5.0
3,1.0,27,48,4.0
4,1.0,27,60,3.0


حالا با امتیازهایی که از فیلم ها داریم و میزار شباهت یک ستون جدید میسازیم که حاوی امتیاز وزن دهی شده است

In [16]:
topUsersRating["weightedRating"] = topUsersRating["similarityIndex"] * topUsersRating["rating"]
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,27,1,3.0,3.0
1,1.0,27,2,4.0,4.0
2,1.0,27,34,5.0,5.0
3,1.0,27,48,4.0,4.0
4,1.0,27,60,3.0,3.0


یک دیتافریم جدید میسازیم که حاوی آیدی هر فیلم و میزان شباهت و امتیاز وزن دهی شده آن میباشد

In [17]:
groupTopUsersRating = topUsersRating.groupby("movieId").sum()[["similarityIndex", "weightedRating"]]
groupTopUsersRating.columns = ["sum_similarityIndex", "sum_weightedRating"]
groupTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,21.050494,79.188182
2,19.972869,69.258831
3,10.120962,32.767503
4,0.777778,2.333333
5,5.650236,17.608639


دیتافریم فیلم های پیشنهادی با استفاده از امتیاز وزن دهی شده میسازیم

In [18]:
rec_df = pd.DataFrame()
rec_df["weighted_rec_score"] = groupTopUsersRating["sum_weightedRating"] / groupTopUsersRating["sum_similarityIndex"]
rec_df["movieId"] = groupTopUsersRating.index
rec_df.head()

,weighted_rec_score,movieId
movieId,,
1,3.761821,1
2,3.467646,2
3,3.237588,3
4,3.000000,4
5,3.116443,5


مرتب سازی برای پیدا کردن فیلم هایی که دارای امتیاز بیشتری دارند

In [19]:
rec_df = rec_df.sort_values(by="weighted_rec_score", ascending=False)
rec_df.head(10)

,weighted_rec_score,movieId
movieId,,
3983,5.0,3983
8008,5.0,8008
928,5.0,928
4499,5.0,4499
88129,5.0,88129
85,5.0,85
82,5.0,82
6993,5.0,6993
3404,5.0,3404


پنج فیلم برتر را استخراج میکنیم

In [20]:
Movies = movie_df.loc[movie_df["movieId"].isin(rec_df.head(5)["movieId"].tolist())]["title"].tolist()
Movies

['Rebecca (1940)',
 'You Can Count on Me (2000)',
 'Dirty Rotten Scoundrels (1988)',
 'Brigadoon (1954)',
 'Drive (2011)']

فیلم های برتر را پیشنهاد میدهیم

In [21]:
print("I recommend to you these movies: \n")
for movie in Movies:
    print("    " + movie, end="\n\n")

I recommend to you these movies: 

    Rebecca (1940)

    You Can Count on Me (2000)

    Dirty Rotten Scoundrels (1988)

    Brigadoon (1954)

    Drive (2011)

